## Описание проекта:
Задача проекта подготовить исследование рынка общественного питания в Москве. На основе этой оценки будет дан прогноз перспективности кафе с роботами в качестве официантов. 

## Используемые библиотеки

In [1]:
import pandas as pd
import plotly.express as px
import re
from io import BytesIO
import requests

## Подготовка данных к анализу

In [2]:
rest_df = pd.read_csv('/Users/polina/Downloads/rest_data.csv')
display(rest_df.head(5))

,id,object_name,chain,object_type,address,number
0,151635,СМЕТАНА,нет,кафе,"город Москва, улица Егора Абакумова, дом 9",48
1,77874,Родник,нет,кафе,"город Москва, улица Талалихина, дом 2/1, корпус 1",35
2,24309,Кафе «Академия»,нет,кафе,"город Москва, Абельмановская улица, дом 6",95
3,21894,ПИЦЦЕТОРИЯ,да,кафе,"город Москва, Абрамцевская улица, дом 1",40
4,119365,Кафе «Вишневая метель»,нет,кафе,"город Москва, Абрамцевская улица, дом 9, корпус 1",50


In [3]:
rest_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15366 entries, 0 to 15365
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           15366 non-null  int64 
 1   object_name  15366 non-null  object
 2   chain        15366 non-null  object
 3   object_type  15366 non-null  object
 4   address      15366 non-null  object
 5   number       15366 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 720.4+ KB


In [4]:
print(rest_df.duplicated(['object_name','chain','object_type','address','number']).sum())

82


In [5]:
rest_df = rest_df.drop_duplicates(subset = ['object_name','chain','object_type','address','number'])

In [6]:
print(rest_df.duplicated(['object_name', 'chain', 'object_type', 'address', 'number']).sum())

0


In [7]:
# приведение названий к нижнему регистру
rest_df['object_name'] = rest_df['object_name'].str.lower()

In [8]:
# убираем слова, которые не нужны в названиях
unneccesary_words = ["кафе", "питания", "ресторан", "«", "»", "закусочная", "бар", "семейный",
                     "предприятие", "быcтрого", "обслуживания", "кальянная", "кальян-бар", 
                    "-", "москва", "лаундж", "детское", "кофейня", "закрытая", ",", "(", ")", "кондитерская",
                     "пекарня", "кондитерия", "пиццерия", "пицерия", "магазин"]
def deleting_words(row):
    row['object_name'] = row['object_name'].split(' ')
    cleared_name = []
    for word in row['object_name']:
        if word not in unneccesary_words:
            cleared_name.append(word)
    cleared_name = ' '.join(cleared_name)     
    return cleared_name
    
rest_df['object_name'] = rest_df.apply(deleting_words, axis = 1) 

### Вывод:

1. У нас есть данные по заведениям в Москве: id, название, является ли сетевым, тип заведения, адрес и количество посадочных мест. 
2. Всего в нашей таблице 15366 уникальных id, но некоторым id соответсвуют полностью повторяющиеся заведения. Эти дубликаты были удалены из таблицы. Явных пропусков в таблице не обнаружено. 
3. Типы данных в столбцах правильные. Но в столбце с названиями заведений все строки были приведены к нижнему регистру и удалены слова типа "кафе" или "ресторан", для дальнейшей обработки сетевых заведений, так как одни и те же заведения при подобных различиях не попадут в одну группу. 

## Анализ данных

### Cоотношение видов объектов общественного питания по количеству

In [9]:
rest_types = (rest_df.groupby('object_type', as_index = False)
              .agg({'id': 'count'})
              .sort_values(by = 'id')
             )
rest_types.columns = ['type', 'number']
display(rest_types)
fig = px.bar(rest_types,
             x = 'type',
             y = 'number',
             labels = dict(type = 'Тип заведения', number = 'Количество'),
             title = 'Количество объектов по типам')
fig.show()

,type,number
5,магазин (отдел кулинарии),273
2,закусочная,349
4,кафетерий,395
1,буфет,577
0,бар,855
6,предприятие быстрого обслуживания,1898
7,ресторан,2282
8,столовая,2584
3,кафе,6071


В имеющимся у нас наборе данных больше всего заведений, юмеющих тип "кафе" (6099 объектов). Такой тип заведений больше, чем в два раза опережает следующий тип "столовая" (2587 объектов). Меньше всего в данных "магазинов с отделом кулинарии" (273 объекта).  

### Cоотношение сетевых и несетевых заведений по количеству

In [10]:
rest_chain = (rest_df.groupby('chain', as_index = False)
              .agg({'id': 'count'})
              .sort_values(by = 'id')
             )
rest_chain.columns = ['chain', 'number']
display(rest_chain)

,chain,number
0,да,2964
1,нет,12320


In [11]:
fig = px.bar(rest_chain,
             x = 'chain',
             y = 'number',
             labels = dict(chain = 'Является ли сетевым', number = 'Количество'),
             title = 'Количество сетевых и несетевых заведений')
fig.show()

Несетевых заведений питания в нашей базе данных примерно в четыре раза больше, чем сетевых (12398 против 2968). 

### Для какого вида объекта общественного питания характерно сетевое распространение?

In [12]:
rest_types_chains = rest_df.groupby(['object_type', 'chain'], as_index = False).agg({'id': 'count'})
display(rest_types_chains)
fig = px.bar(rest_types_chains,
             x = 'object_type',
             y = 'id', 
             color = 'chain',
             title = 'Соотношение сетевых и несетевых заведений в каждом типе заведения',
             labels = dict(object_type = 'Типы объектов', id = 'Количество', chain = ' Является ли сетевым'))
fig.show()

,object_type,chain,id
0,бар,да,37
1,бар,нет,818
2,буфет,да,11
3,буфет,нет,566
4,закусочная,да,56
5,закусочная,нет,293
6,кафе,да,1396
7,кафе,нет,4675
8,кафетерий,да,52
9,кафетерий,нет,343


Был построен график соотношения количества сетевых и несетевых заведений по каждому типу. Типом заведения с самой большой долей сетевых предприятий является "предприятия быстрого обслуживания", у которых почти половина заведений сетевые. Затем следуют "кафе" и "ресторан". Для всех остальных типов заведений не характерен сетевой формат. 

### Что характерно для сетевых заведений: много заведений с небольшим числом посадочных мест в каждом или мало заведений с большим количеством посадочных мест?

***Какое количество посадочных мест считать границей между "много" и "мало"?***

In [13]:
chain_rests = rest_df.query('chain == "да"')
fig = px.box(chain_rests,
             y = 'number',
             title = 'Расмпределение количества посадочных мест в сетевых заведениях')
fig.show()

В качестве границы будет взято медианное значение. То есть, если посадочных мест больше 40, то отнесем эти заведения к категории "много посадочных мест", а все остальные к "мало посадочных мест".

***Какое число заведений у сети считать "большим", а какое "маленьким"?***

In [14]:
chain_rests_groupped = (chain_rests.groupby('object_name', as_index = False)
                        .agg({'id': 'nunique'})
                        .sort_values(by = 'id', ascending = False)
                       )

In [15]:
fig = px.box(chain_rests_groupped,
             y = 'id',
             title = 'Распределение количества точек в сетевых заведениях')
fig.show()

Так как в наших данных с названиями заведений даже после предварительной обработки все равно остались скрытые дубликаты, при оценке количества точек у сети, мы возьмем значение выше, чем медиана. Границей между "много точек" и "мало точек" будет 8.

***Распредение заведений на две группы***

In [16]:
seat_spots_groupped = (chain_rests.groupby('object_name')
                       .agg({'number': 'median', 'id': 'count'})
                      )
seat_spots_groupped.columns = ['seats', 'spots']

In [17]:
fig = px.scatter(seat_spots_groupped,
                 x = 'spots',
                 y = 'seats',
                 title = 'Распределение сетевых ресторанов по количеству посадочных мест и количеству объектов')
fig.add_hline(y=40, line_width=3, line_dash="dash", line_color="green")
fig.add_vline(x=8, line_width=3, line_dash="dash", line_color="green")
fig.show()

Были определены границы оценки количества точек у сети и количества посадочных мест. И построен график рассеяния для количества точек у сети и среднего количества посадочных мест в заведениях этой сети. Мы можем видеть, что большинство значений на графике расположены до границы большого количества точек (8 заведений), но выходят за границу по посадочным местам (40 мест). А ресторанных сетей, у которых мало посадочных мест, но много точек, значительно меньше. Поэтому можно сделать вывод, что для сетевых заведений характерно иметь много точек, но рассчитанных на небольшое количество людей. 

### Опишите среднее количество посадочных мест. Какой вид предоставляет в среднем самое большое количество посадочных мест?

In [18]:
# среднее количество посадочных мест по всем типам заведений

print(rest_df['number'].mean())

59.78434964668935


In [19]:
seats_groupped = (rest_df.groupby('object_type', as_index= False)
                  .agg({'number': 'median'})
                  .sort_values(by = 'number', ascending = False)
                 )
seats_groupped

,object_type,number
8,столовая,103.0
7,ресторан,80.0
0,бар,35.0
1,буфет,32.0
3,кафе,30.0
4,кафетерий,6.0
6,предприятие быстрого обслуживания,6.0
2,закусочная,0.0
5,магазин (отдел кулинарии),0.0


In [20]:
fig = px.bar(seats_groupped,
             x = 'object_type',
             y = 'number',
             title = 'Среднее количество посадочных мест по типу заведения',
             labels = dict(object_type = 'Типы объектов', number = 'Среднее количество посадочных мест')
            )
fig.show()

Была создана таблица и наглядый график медианного количества посадочных мест в каждом типе заведения (и сетевых и несетевых). Самое большое количество посадочных мест в столовых, что легко объясняется их спецификой. Чаще всего столовые находятся, либо при каком-либо предприятии, либо в непосредственной близости к ним, и должны выдерживать наплыв своей основной целевой аудитории, работающих людей, в часы обеда. Меньше всего посадочных мест в магазинных отделах кулинарии. Вероятно, магазины специально не делают большое количество посадочных мест, чтобы клиенты реже решали есть прям в магазине, чтобы избежать издержек на обслуживание и уборку после клиентов.

### Топ-10 улиц по количеству объектов общественного питания

In [21]:
# создание столбца с названиями улиц 
words = ['улица','ул','переулок','шоссе','проспект','площадь','проезд',
         'село','аллея','бульвар','набережная','тупик','линия']

str_pat = r".*,\s*\b([^,]*?(?:{})\b[^,]*)[,$]+".format("|".join(words))

rest_df["street"] = rest_df["address"].str.extract(str_pat, flags=re.I)
rest_df.head(3)

,id,object_name,chain,object_type,address,number,street
0,151635,сметана,нет,кафе,"город Москва, улица Егора Абакумова, дом 9",48,улица Егора Абакумова
1,77874,родник,нет,кафе,"город Москва, улица Талалихина, дом 2/1, корпус 1",35,улица Талалихина
2,24309,«академия»,нет,кафе,"город Москва, Абельмановская улица, дом 6",95,Абельмановская улица


In [22]:
# таблица с количеством заведений по улицам
streets_groupped = (rest_df.groupby('street', as_index=False)
                    .agg({'id': 'nunique'})
                    .sort_values(by = 'id', ascending= False)
                   )
streets_groupped.columns = ['street', 'number_of_spots']
top_streets_groupped = streets_groupped.head(10)
top_streets_groupped

,street,number_of_spots
1418,проспект Мира,204
1009,Профсоюзная улица,181
683,Ленинградский проспект,170
993,Пресненская набережная,167
397,Варшавское шоссе,164
686,Ленинский проспект,147
1414,проспект Вернадского,128
673,Кутузовский проспект,114
596,Каширское шоссе,111
603,Кировоградская улица,108


In [23]:
fig = px.pie(top_streets_groupped,
             values = 'number_of_spots',
             names = 'street', 
             title = 'Топ-10 улиц по количеству заведений')
fig.show()

In [24]:
# Таблица с улицами и их районами

spreadsheet_id = '1d0ZjAXd2OI-enT6vz6-G2s07avtiu6oQ0hIy39mERBU'
file_name = 'https://docs.google.com/spreadsheets/d/{}/export?format=csv'.format(spreadsheet_id)
r = requests.get(file_name)
districts = pd.read_csv(BytesIO(r.content))
districts.columns = ['street', 'area_id', 'area', 'district']
districts.head(5)

,street,area_id,area,district
0,Выставочный переулок,17,ЦАО,Пресненский район
1,улица Гашека,17,ЦАО,Пресненский район
2,Большая Никитская улица,17,ЦАО,Пресненский район
3,Глубокий переулок,17,ЦАО,Пресненский район
4,Большой Гнездниковский переулок,17,ЦАО,Пресненский район


In [25]:
# соединение таблицы с количеством завдений по улицам с их районами
top_streets_groupped = top_streets_groupped.merge(districts[['street','district']],
                                          on = 'street',
                                          how = 'left')
top_streets_groupped.head(10)

,street,number_of_spots,district
0,проспект Мира,204,Алексеевский район
1,проспект Мира,204,Ярославский Район
2,проспект Мира,204,Район Марьина роща
3,проспект Мира,204,Останкинский район
4,проспект Мира,204,Район Ростокино
5,проспект Мира,204,Район Свиблово
6,проспект Мира,204,Мещанский район
7,Профсоюзная улица,181,Академический район
8,Профсоюзная улица,181,Район Черемушки
9,Профсоюзная улица,181,Район Ясенево


In [26]:
# количество заведений по районам
districts_groupped = (top_streets_groupped.groupby('district', as_index=False)
                    .agg({'number_of_spots': 'sum'})
                    .sort_values(by = 'number_of_spots', ascending= False)
                    .head(10)
                    )
display(districts_groupped)

,district,number_of_spots
25,Район Теплый Стан,328
6,Обручевский район,328
2,Донской район,311
19,Район Проспект Вернадского,275
3,Ломоносовский район,275
16,Район Нагатино-Садовники,275
26,Район Тропарево-Никулино,275
11,Район Гагаринский,275
30,Район Чертаново Центральное,272
29,Район Чертаново Северное,272


In [27]:
fig = px.pie(districts_groupped,
             values = 'number_of_spots',
             names = 'district',
             title = 'Топ-10 районов по количеству заведений'
            )
fig.show()

Была создана таблица с числом точек общественного питания по улицам Москвы. Ултцей с самым большим числом заведений стал проспект Мира. Если взглянуть на список топ-10 таких улиц, общая черта всех этих улиц: большая протяженность. Все они проходят сразу через много районов. И, естественно, имеют больше заведений, чем очень короткие улицы.  Если взглянуть на таблицу с количеством заведений по районам, то самый популярный район Тверской. Несмотря на то, что самая популярная улица расположена не в этом районе, Тверской имеет множество маленьких улиц с большой концентрацией заведений, но отдельно взятые улицы Тверского района чисто физически не могут уместить больше, чем проспект Мира. Также можно заметить, что почти все улицы находятся в ЦАО. 

### Число улиц с одним объектом общественного питания

In [28]:
streets_with_1_spot = streets_groupped.query('number_of_spots == 1')
print(streets_with_1_spot.info())
display(streets_with_1_spot.head(10))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 551 entries, 171 to 952
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   street           551 non-null    object
 1   number_of_spots  551 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 12.9+ KB
None


,street,number_of_spots
171,3-я Песчаная улица,1
139,2-я Филёвская улица,1
169,3-я Новоостанкинская улица,1
149,3-й Кадашёвский переулок,1
876,Новопресненский переулок,1
1706,улица Медведева,1
1029,Рижский проезд,1
150,3-й Кожуховский проезд,1
1707,улица Медиков,1
148,3-й Дорожный проезд,1


In [29]:
streets_with_1_spot = streets_with_1_spot.merge(districts[['street','district']],
                                          on = 'street',
                                          how = 'left')
streets_with_1_spot = (streets_with_1_spot.groupby('district')
                       .agg({'street': 'nunique'})
                       .sort_values(by = 'street', ascending = False)
                       .head(10)
                        )
streets_with_1_spot

,street
district,
Таганский район,27
Район Хамовники,26
Басманный район,25
Тверской район,20
Пресненский район,19
Район Марьина роща,18
Мещанский район,17
Район Арбат,17
Район Замоскворечье,14


Всего в нашей базе данных 598 улиц, на которых только одно заведение общественного питания. Больше всего таких улиц в Таганском районе. Но почти все районы из данного списка входят и в список с самым большим количеством заведений по районам. 

## Вывод: 

1. Был проведен анализ, имеющихся у нас данных по сетевым и несетевым заведениям общественного питания в Москве на основе типа заведений, их расположения и количества посадочных мест. Основная проблема с имеющимися данными азключалась в названиях заведений. Одни и те же названия могли быть записаны с незначительным отличием, что при дальнейшей группировке, давало бы неявные дубликаты. Поэтому названия были приведены к максимально возможному соответствию. 
2. При анализе типов заведений, мы увидели, что самым популярным типом является "кафе". Это может быть обусловлено тем, что эта категория имеет самые рамытые рамки, то есть любое заведение без каких-либо особенностей может в нее попасть. И "кафе" вероятней всего самый выгодный тип из-за среднего ценового диапозона, и имеет самую большую целевую аудиторию. Поэтому 
3. Несетевых заведений в Москве больше, чем сетевых. 
4. Количество посадочных мест не имеет корреляции с количеством точек сетевых заведений. У сетей с самым большим количеством точек действительно в среднем не больше 100 посадочных мест. Но у сетей с относительно небольшим количеством точек нельзя выявить точный тренд, количество посадочных мест распределено примерно одинаково. При планировании дальнейшего расширения сети кафе с роботами не стоит уменьшать количество посадочных мест, и стремиться к большому количеству точек, это затратнее. 
5. Если рассмотреть средние показатели посадочных мест по типу заведений, то можно увидеть, что в "столовых" их больше всего. Это делается для того6 чтобы выдерживать неравномернве наплывы посетителей в течении дня. Меньше всего посадочных мест в магазинных отделах кулинарии, закусочных и предприятиях быстрого обслуживания, чтобы была большая проходимость, поточность обслуживания и, как следствие, уменьшение затрат. При планировании посадочных мест в будущем кафе лучше всего взять среднее занчение для этого типа заведений, и сделать около 30 посадочных мест. Посадочные места следует учитывать при закупке роботов-официантов, чтобы их хвало для обслуживание 30 посадочных мест. 
6. Были рассматренны концентрацию заведений общественого питания по улицам и районам Москвы. При распределении по улицам, самая большая концентрация заведений на проспекте Мира, Ленинградском и Профсоюзной улице. Эти в все остальные улицы из топ-10 проходят сразу через несколько районов, и берут первенство за счет своей большой протяженности. Самая большая концентрация точек общественного питания в Тверском,  Пресненском и Басманном районах. Все эти районы расположены в центре Москвы. И берут первенство за счет относительно небольшого количества заведений почти на каждой улице этих районов. Если выбирать район для будущего заведения, то стоит выбирать из топ-10, но не лидеров. В Теврском, Пресненском и Басманном районах несмотря на высокую проходимость, самая высокая конкуренция. Поэтому лучше выбрать между Даниловским, Мещанским и Донским районами. 

Презентация: <https://drive.google.com/file/d/1gj-kAIyFzONhBzZyAZ2rxrZo3Li3NfU2/view?usp=sharing> 